In [63]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline

In [64]:
cap = cv.VideoCapture('content/corgi_race.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    frame = cv.resize(frame, (640, 480))
    #Conversáo para escala de cinza
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # calculate shifted dft
    dft = cv.dft(np.float32(frame),flags = cv.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    real = dft_shift[:,:,0]
    imag = dft_shift[:,:,1]

    # calculate spectrum
    spectrum = 20*np.log(cv.magnitude(real,imag)+0.0001)
    spectrum_norm = spectrum/np.max(spectrum)

    # show grayscale frame and corresponding spectrum
    cv.imshow('spectrum', spectrum_norm)
    cv.imshow('frame', frame)

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...


# 1. Implemente um filtro passa-baixa ideal, aplique o filtro à transformada de Fourier do vídeo e mostre o vídeo do espectro resultantes para diferentes valores de D0.

In [65]:
def create_mask_PB_ideal(rows, cols, radius):
    center = (int(cols / 2), int(rows / 2))
    mask = np.zeros((rows, cols), np.uint8)
    cv.circle(mask, center, radius, 1, thickness=-1)
    return mask

In [66]:
D0_values = [30, 60, 100]

for D0 in D0_values:
    cap = cv.VideoCapture('content/corgi_race.mp4')

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        
        frame = cv.resize(frame, (640, 480))
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # calculate shifted dft
        dft = cv.dft(np.float32(frame), flags=cv.DFT_COMPLEX_OUTPUT)
        dft_shift = np.fft.fftshift(dft)

        real = dft_shift[:,:,0]
        imag = dft_shift[:,:,1]

        # calculate spectrum
        spectrum = 20 * np.log(cv.magnitude(real, imag) + 0.0001)
        spectrum_norm = spectrum / np.max(spectrum)
        
        # create ideal low-pass filter
        mask = create_mask_PB_ideal(frame.shape[0], frame.shape[1], D0)
        new_spectrum = spectrum_norm * mask

        # apply mask
        mask = np.stack((mask, mask), axis=-1)
        dft_shift_filtered = dft_shift * mask

        # inverse DFT
        f_ishift = np.fft.ifftshift(dft_shift_filtered)
        img_back = cv.idft(f_ishift)
        img_back = cv.magnitude(img_back[:,:,0], img_back[:,:,1])

        # normalize the image
        cv.normalize(img_back, img_back, 0, 1, cv.NORM_MINMAX)

        # show grayscale frame and corresponding spectrum
        cv.imshow(f'spectrum (D0={D0})', new_spectrum)
        cv.imshow(f'frame (D0={D0})', img_back)

        if cv.waitKey(1) == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...
Can't receive frame (stream end?). Exiting ...
Can't receive frame (stream end?). Exiting ...


# 2. Implemente um filtro passa-baixa de Butterworth, aplique o filtro à transformada de Fourier do vídeo e mostre o vídeo do espectro resultantes para diferentes valores de D0 e de n.

In [67]:
def butterworth_lowpass_filter(rows, cols, D0, n):
    u = np.arange(0, cols)
    v = np.arange(0, rows)
    u, v = np.meshgrid(u, v)
    D = np.sqrt((u - cols/2)**2 + (v - rows/2)**2)
    H = 1 / (1 + (D / D0)**(2 * n))
    return H

In [68]:
D0_values = [30, 50, 70]
n_values = [1, 2, 3]

for D0 in D0_values:
    for n in n_values:
        cap = cv.VideoCapture('content/corgi_race.mp4')

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Can't receive frame (stream end?). Exiting ...")
                break
            
            frame = cv.resize(frame, (640, 480))
            frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

            # calculate shifted dft
            dft = cv.dft(np.float32(frame), flags=cv.DFT_COMPLEX_OUTPUT)
            dft_shift = np.fft.fftshift(dft)

            real = dft_shift[:,:,0]
            imag = dft_shift[:,:,1]

            # calculate spectrum
            spectrum = 20 * np.log(cv.magnitude(real, imag) + 0.0001)
            spectrum_norm = spectrum / np.max(spectrum)
            
            # create Butterworth low-pass filter
            mask = butterworth_lowpass_filter(frame.shape[0], frame.shape[1], D0, n)

            new_spectrum = spectrum_norm * mask

            # apply mask
            mask = np.stack((mask, mask), axis=-1)
            dft_shift_filtered = dft_shift * mask


            # inverse DFT
            f_ishift = np.fft.ifftshift(dft_shift_filtered)
            img_back = cv.idft(f_ishift)
            img_back = cv.magnitude(img_back[:,:,0], img_back[:,:,1])

            # normalize the image
            cv.normalize(img_back, img_back, 0, 1, cv.NORM_MINMAX)

            # show grayscale frame and corresponding spectrum
            cv.imshow(f'spectrum (D0={D0}e n={n})', new_spectrum)
            cv.imshow(f'frame (D0={D0} e n={n})', img_back)

            if cv.waitKey(1) == ord('q'):
                break

        cap.release()
        cv.destroyAllWindows()

Can't receive frame (stream end?). Exiting ...
Can't receive frame (stream end?). Exiting ...
Can't receive frame (stream end?). Exiting ...


# 3. Implemente um filtro passa-alta gaussiano, aplique o filtro à transformada de Fourier do vídeo e mostre o vídeo do espectro resultantes para diferentes valores de D0.

In [69]:
def gaussian_highpass_filter(rows, cols, D0):
    u = np.arange(0, cols)
    v = np.arange(0, rows)
    u, v = np.meshgrid(u, v)
    D = np.sqrt((u - cols/2)**2 + (v - rows/2)**2)
    H = 1 - np.exp(- (D**2) / (2 * (D0**2)))
    return H

In [70]:
D0_values = [30, 60, 100]

for D0 in D0_values:
    cap = cv.VideoCapture('content/corgi_race.mp4')

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print(f"Can't receive frame with D0={D0} (stream end?). Exiting ...")
            break
        
        frame = cv.resize(frame, (640, 480))
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        # calculate shifted dft
        dft = cv.dft(np.float32(frame), flags=cv.DFT_COMPLEX_OUTPUT)
        dft_shift = np.fft.fftshift(dft)

        real = dft_shift[:,:,0]
        imag = dft_shift[:,:,1]

        # calculate spectrum
        spectrum = 20 * np.log(cv.magnitude(real, imag) + 0.0001)
        spectrum_norm = spectrum / np.max(spectrum)
        
        # create Gaussian high-pass filter
        mask = gaussian_highpass_filter(frame.shape[0], frame.shape[1], D0)

        new_spectrum = spectrum_norm * mask

        # apply mask
        mask = np.stack((mask, mask), axis=-1)
        dft_shift_filtered = dft_shift * mask

        # inverse DFT
        f_ishift = np.fft.ifftshift(dft_shift_filtered)
        img_back = cv.idft(f_ishift)
        img_back = cv.magnitude(img_back[:,:,0], img_back[:,:,1])

        # normalize the image
        cv.normalize(img_back, img_back, 0, 1, cv.NORM_MINMAX)

        # show grayscale frame and corresponding spectrum
        cv.imshow(f'spectrum (D0={D0})', new_spectrum)
        cv.imshow(f'frame (D0={D0})', img_back)

        if cv.waitKey(1) == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()